In [ ]:
import wandb
wandb.login()

In [ ]:
import gc
import os
import sys
sys.path.append('../')
import time
from collections import namedtuple
from tqdm import tqdm
import numpy as np
import torch
import torchvision
import src.utils as utils
from src.models import LeNetMNIST, ResVestimatorMNIST, R18VestimatorMNIST, Vestimator, LeVestimatorMNIST
from src.frameworks.online_dvrl import Odvrl
from src.frameworks.online_proposed import Proposed

In [ ]:
from datetime import datetime
experiment_time = datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d--%H:%M:%S.%f')
experiment_time

In [ ]:
from collections import namedtuple
Parameters = namedtuple('Parameters', [
    'saving_path',
    'val_batch_size',  # 不用调整，除非cuda内存爆了
    'epochs',  # RL线下学习量
    'device',  # 一般设'cuda'
    'vest_learning_rate',  # 估价网络的学习率
    'pred_learning_rate',  # 训练用分类器的学习率
    'num_workers',  # 调大可能能运行快一些
    'explore_strategy',  # RL探索策略，可以选'constant', 'linear', 'exponential'
    'epsilon0',  # [0,1]区间内的值，初始探索要求，值越大探索越多
    'threshold',  # 对我们提出的算法(proposed)没用，对DVRL有用，控制DVRL的探索，值越大探索越多
    'discover_record_interval',  # 每几步RL迭代要检测discover rate
    'is_debug',  #  是否在debug
    'vest_lr_scheduler',  # 估价网络的学习scheduler
])

In [ ]:
parameters = Parameters(
    saving_path='../logs',
    val_batch_size=1024,
    epochs=100,
    device='cpu',
    vest_learning_rate=1e-6,
    pred_learning_rate=1e-6,
    num_workers=3,
    explore_strategy='constant',  # ['linear', 'constant', 'exponential']
    epsilon0=0.5,
    threshold=0.5,
    discover_record_interval=1,
    is_debug=True,
    vest_lr_scheduler='exponential',
)

In [ ]:
T = 1  # 在线实验的时间长度，调小一些可以运行快一点，找比较好的参数的时候可以先调小T
noise_level = 0.2  # 噪音数据的含量
seed = 5  # 随机种子
num_weak = 200  # 控制OOB的精度，越大OOB越好，可以跳到500
pred_model = LeNetMNIST()
val_model = LeNetMNIST()
value_estimator = LeVestimatorMNIST()

In [ ]:
wandb.init(
    # Set the project where this run will be logged
    project="online-noise-detection-proposed", 
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name=f"experiment_"+experiment_time, 
    # Track hyperparameters and run metadata
    config={
    "T": T,
    "noise_level": noise_level,
    "seed": seed,
    "num weak learner": num_weak,
    "val_batch_size": parameters.val_batch_size,
    "epochs": parameters.epochs,
    "vest_learning_rate": parameters.vest_learning_rate,
    "pred_learning_rate": parameters.pred_learning_rate,
    "num workers": parameters.num_workers,
    "explore_strategy": parameters.explore_strategy,
    "epsilon0": parameters.epsilon0,
    "threshold": parameters.threshold,
    "discover_record_interval": parameters.discover_record_interval,
    "value estimator architecture": "LeVestimatorMNIST",
    "pred model architecture": "LeNetMNIST",
    "validation model architecture": "LeNetMNIST",
    "dataset": "MNIST",
})

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model.classify(data)
        loss = torch.nn.functional.nll_loss(output, target)
        loss.backward()
        optimizer.step()

In [ ]:
transform=torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.1307,), (0.3081,))
])

In [ ]:
if os.path.isfile('../data/pretrained_models/pretrained_levestimator.pt'):
    value_estimator.load_state_dict(
        torch.load(
            '../data/pretrained_models/pretrained_levestimator.pt',
            map_location=torch.device(parameters.device)
        )
    )
else:
    train_dataset = torchvision.datasets.MNIST('../data', train=True, download=True, transform=transform)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
    optimizer = torch.optim.Adadelta(value_estimator.parameters(), lr=1.0)
    value_estimator.to('cuda')
    for epoch in tqdm(range(14)):
        train(value_estimator, 'cuda', train_loader, optimizer, epoch)
    torch.save(
        value_estimator.state_dict(),
        '../data/pretrained_models/pretrained_levestimator.pt'
    )

In [ ]:
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
# run 'pretrain_mnist.ipynb' first
# then load this pretrained model
state_dict = torch.load("../data/pretrained_models/pretrained_lenetmnist.pt", map_location=torch.device(parameters.device))
val_model.load_state_dict(state_dict)

In [ ]:
x_train, y_train, x_test, y_test, noisy_idxs = utils.create_noisy_mnist(method='uniform', noise_level=noise_level)

In [ ]:
noisy_idxs.sort()
noisy_idxs[:20]

In [ ]:
x_train = torch.tensor(x_train)
y_train = torch.tensor(y_train)
test_data = torchvision.datasets.MNIST('../data', train=False, download=True, transform=transform)

In [ ]:
num_data = len(y_train)

In [ ]:
engine = Proposed(num_weak=num_weak, pred_model=pred_model, val_model=val_model, value_estimator=value_estimator, parameters=parameters)

In [ ]:
time.asctime()

In [ ]:
subset_len = num_data // T
for t in range(T):
    start_id = t * subset_len
    end_id = min((t + 1) * subset_len, num_data)
    # DEBUG
    end_id = 2000
    current_noisy_idxs = np.extract((noisy_idxs >= start_id) & (noisy_idxs < end_id), noisy_idxs)
    current_corrupted_num = len(current_noisy_idxs)
    engine.one_step(
        t, 
        X=x_train[start_id:end_id], 
        y=y_train[start_id:end_id], 
        val_dataset=test_data, 
        subset_len=subset_len, 
        corrupted_num=current_corrupted_num,
        noisy_idxs=current_noisy_idxs-start_id,
        discover_record_interval=parameters.discover_record_interval,
    )

In [ ]:
time.asctime()